<a href="https://colab.research.google.com/github/usrmaia/Mundo-de-Wumpus/blob/main/Mundo_de_Wumpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from random import random
!pip install tabulate # Exibe dados em uma grid
from tabulate import tabulate
!pip install colorama # Dar cor ao texto
from colorama import Fore, init, deinit
!pip install python-sat
from pysat.solvers import Glucose3
!pip install db-sqlite3
import sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Geração do Mapa
Para cada posição temos **((wumpus, poco, ouro), (fedor, brisa, brilho, impacto, grito))** atribuído de forma aleatória.

Esse dado (tupla) será captado pelo sensor.
### Exemplos:
* Posição segura: ((0, 0, 0), (0, 0, _, _, _))
* Posição com 3 poços adjacentes: ((0, 0, 0), (_, 3, _, _, _))

## Declaração
mapa = Mapa(linhas, colunas, wumpus, pocos, ouros)

In [3]:
class Mapa():
  def __init__(self, linhas, colunas, quantidade_wumpus, quantidade_pocos, quantidade_ouros):
    self.linhas = linhas
    self.colunas = colunas
    self.quantidade_wumpus = quantidade_wumpus
    self.quantidade_pocos = quantidade_pocos
    self.quantidade_ouros = quantidade_ouros
    self.taxa_wumpus = self.quantidade_wumpus / (self.linhas * self.colunas - 1 - self.quantidade_pocos - self.quantidade_ouros)
    self.taxa_pocos = self.quantidade_pocos / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_ouros)
    self.taxa_ouros = self.quantidade_ouros / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_pocos)
    self.status_posicao = {} # linha_coluna_posicao:((wumpus, poco, ouro), (fedor, brisa, brilho, impacto, grito))
    self.mapa = []
    self.mapearStatus(self)

  @staticmethod
  def mapearStatus(self):
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        self.status_posicao[f"{linha}_{coluna}_posicao"] = ((0, 0, 0), (0, 0, 0, 0, 0))
  
  def gerarMapa(self):
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        if self.insercaoValida(linha, coluna):
          self.adicionarWumpusEFedor(self, linha, coluna)
          self.adicionarPocoEBrisa(self, linha, coluna)
          self.adicionarOuroEBrilho(self, linha, coluna)
        self.adicionarImpacto(self, linha, coluna)

  def imprimirMapa(self):
    init() # colorama
    for linha in range(0, self.linhas):
      elementos_da_linha = []
      for coluna in range(0, self.colunas):
        elementos_da_celula = ""
        if self.possuiWumpus(self, linha, coluna):
          elementos_da_celula += (Fore.GREEN + "wumpus")
        if self.possuiPoco(self, linha, coluna):
          elementos_da_celula += (Fore.RED + "poco")
        if self.possuiBrilho(self, linha, coluna):
          elementos_da_celula += (Fore.YELLOW + "ouro")
        if self.possuiFedor(self, linha, coluna):
          elementos_da_celula += (Fore.GREEN + str(self.possuiFedor(self, linha, coluna)) + "fedor")
        if self.possuiBrisa(self, linha, coluna):
          elementos_da_celula += (Fore.RED + str(self.possuiBrisa(self, linha, coluna)) + "brisa")
        if self.possuiOK(self, linha, coluna):
          elementos_da_celula += (Fore.WHITE + "ok")
      
        elementos_da_linha.append(elementos_da_celula)
      self.mapa.append(elementos_da_linha)
    deinit() # colorama
    print(tabulate(self.mapa, tablefmt="plain", stralign="center"))
    print(Fore.WHITE) # gambiarra
  
  def retornarStatusDaPosica(self, linha, coluna):
    try:
      return self.status_posicao[f"{linha}_{coluna}_posicao"]
    except:
      print("Posição inválida!")
      return False

  def pegarOuro(self, linha, coluna):
    ouru = self.status[f"{linha}_{coluna}_pos"][0][2]
    if ouro:
      status = self.status[f"{linha}_{coluna}_pos"]
      status = ((status[0][0], status[0][1], status[0][2] - 1), (status[1][0], status[1][1], status[1][2] + 1, status[1][3]))
      self.status[f"{linha}_{coluna}_pos"] = status
      return True
    return False

  @staticmethod
  def insercaoValida(linha, coluna):
    return not ((linha == 0 and coluna == 0))

  @staticmethod
  def adicionarWumpusEFedor(self, linha, coluna):
    if self.quantidade_wumpus >= 1:
      if random() < self.taxa_wumpus:
        self.adicionarWumpus(self, linha, coluna)
        self.adicionarFedor(self, linha - 1, coluna)
        self.adicionarFedor(self, linha, coluna - 1)
        self.adicionarFedor(self, linha + 1, coluna)
        self.adicionarFedor(self, linha, coluna + 1)
        self.quantidade_wumpus -= 1
  
  @staticmethod
  def adicionarPocoEBrisa(self, linha, coluna):
    if self.quantidade_pocos >= 1 and not self.possuiWumpus(self, linha, coluna):
      if random() < self.taxa_pocos:
        self.adicionarPoco(self, linha, coluna)
        self.incrementaBrisa(self, linha - 1, coluna)
        self.incrementaBrisa(self, linha, coluna - 1)
        self.incrementaBrisa(self, linha + 1, coluna)
        self.incrementaBrisa(self, linha, coluna + 1)
        self.quantidade_pocos -= 1
  
  @staticmethod
  def adicionarOuroEBrilho(self, linha, coluna):
    if self.quantidade_ouro >= 1 and not (self.possuiWumpus(self, linha, coluna) or self.possuiPoco(self, linha, coluna)):
      if random() < self.taxa_ouro:
        self.adicionarOuro(self, linha, coluna)
        self.incrementaBrilho(self, linha, coluna)
        self.quantidade_ouro -= 1
  
  @staticmethod
  def adicionarImpacto(self, linha, coluna):
    if (linha == 0 or linha == self.linhas - 1) and (coluna == 0 or coluna == self.colunas - 1):
      self.incrementaImpacto(self, linha, coluna)

  @staticmethod
  def adicionarWumpus(self, linha, coluna):
    status = self.status[f"{linha}_{coluna}_pos"]
    status = ((status[0][0] + 1, status[0][1], status[0][2]), (status[1]))
    self.status[f"{linha}_{coluna}_pos"] = status

  @staticmethod
  def adicionarFedor(self, linha, coluna):
    try:
      status = self.status[f"{linha}_{coluna}_pos"]
      status = ((status[0]), (status[1][0] + 1, status[1][1], status[1][2], status[1][3]))
      self.status[f"{linha}_{coluna}_pos"] = status
    except: pass
  
  @staticmethod
  def possuiWumpus(self, linha, coluna):
    if self.status[f"{linha}_{coluna}_pos"][0][0]:
      return True
    return False

  @staticmethod
  def adicionarPoco(self, linha, coluna):
    status = self.status[f"{linha}_{coluna}_pos"]
    status = ((status[0][0], status[0][1] + 1, status[0][2]), (status[1]))
    self.status[f"{linha}_{coluna}_pos"] = status

  @staticmethod
  def incrementaBrisa(self, linha, coluna):
    try:
      status = self.status[f"{linha}_{coluna}_pos"]
      status = ((status[0]), (status[1][0], status[1][1] + 1, status[1][2], status[1][3]))
      self.status[f"{linha}_{coluna}_pos"] = status
    except: pass
  
  @staticmethod
  def possuiPoco(self, linha, coluna):
    if self.status[f"{linha}_{coluna}_pos"][0][1]:
      return True
    return False
  
  @staticmethod
  def adicionarOuro(self, linha, coluna):
    status = self.status[f"{linha}_{coluna}_pos"]
    status = ((status[0][0], status[0][1], status[0][2] + 1), (status[1]))
    self.status[f"{linha}_{coluna}_pos"] = status
  
  @staticmethod
  def incrementaBrilho(self, linha, coluna):
    status = self.status[f"{linha}_{coluna}_pos"]
    status = ((status[0]), (status[1][0], status[1][1], status[1][2] + 1, status[1][3])) 
    self.status[f"{linha}_{coluna}_pos"] = status

  @staticmethod
  def incrementaImpacto(self, linha, coluna):
    status = self.status[f"{linha}_{coluna}_pos"]
    status = ((status[0]), (status[1][0], status[1][1], status[1][2], status[1][3] + 1, status[1][4])) 
    self.status[f"{linha}_{coluna}_pos"] = status
  
  @staticmethod
  def possuiFedor(self, linha, coluna):
    fedor = self.status[f"{linha}_{coluna}_pos"][1][0]
    if fedor:
      return fedor
    return False
  
  @staticmethod
  def possuiBrisa(self, linha, coluna):
    brisa = self.status[f"{linha}_{coluna}_pos"][1][1]
    if brisa:
      return brisa
    return False
  
  @staticmethod
  def possuiBrilho(self, linha, coluna):
    if self.status[f"{linha}_{coluna}_pos"][1][2]:
      return True
    return False
  
  @staticmethod
  def possuiOK(self, linha, coluna):
    status = self.status[f"{linha}_{coluna}_pos"]
    for tupla in status:
      for i in tupla:
        if i != 0:
          return False
    return True

In [ ]:
mapa = Mapa(4, 4, 1, 2, 2)
mapa.imprimirMapa()

ok  ouro  ouro1brisa   poco
ok   ok       ok      1brisa
ok   ok       ok        ok
ok   ok       ok        ok



# Geração do Mapa do Conhecimento
Mapa gerado a partir da base de conhecimento (linguagem natural)

In [ ]:
class MapaConhecimento():
    def __init__(self):
        self.conhecimento = []
        self.mapa = []
        self.linhas = 0
        self.colunas = 0
    
    def inserirConhecimento(self, conhecimento):
        self.conhecimento = conhecimento
        self.descobrirLinhaEColunaMaximo(self)
        
    def imprimirMapa(self):
      init() # colorama
      for linha in range(0, self.linhas):
        elementos_da_linha = []
        for coluna in range(0, self.colunas):
          elementos_da_celula = ""
          if f"{linha}_{coluna}_wumpus" in self.conhecimento:
            elementos_da_celula += (Fore.GREEN + "wumpus")
          if f"{linha}_{coluna}_poco" in self.conhecimento:
            elementos_da_celula += (Fore.RED + "poco")
          if f"{linha}_{coluna}_ouro" in self.conhecimento:
            elementos_da_celula += (Fore.YELLOW + "ouro")
          if f"{linha}_{coluna}_fedor" in self.conhecimento:
            elementos_da_celula += (Fore.GREEN + "fedor")
          if f"{linha}_{coluna}_brisa" in self.conhecimento:
            elementos_da_celula += (Fore.RED + "brisa")
          if f"{linha}_{coluna}_brilho" in self.conhecimento:
            elementos_da_celula += (Fore.YELLOW + "brilho")
          if f"{linha}_{coluna}_impacto" in self.conhecimento:
            elementos_da_celula += (Fore.BLACK + "parede")
        self.mapa.append(elementos_da_linha)
      deinit() # colorama
      print(tabulate(self.mapa, tablefmt="plain", stralign="center"))
      print(Fore.WHITE) # gambiarra
    
    @staticmethod
    def descobrirLinhaEColunaMaximo(self):
        conhecimento = self.conhecimento
        linhas = 0
        colunas = 0
        for instancia in conhecimento:
          linha, coluna = self.retornarLinhaEColuna(self, instancia)
          if linha > self.linhas: self.linhas = linha
          if coluna > self.coluna: self.coluna = coluna

    @staticmethod
    def retornarLinhaEColuna(self, instancia):
        index = 0
        linha = ""
        while(instancia[index] != "_"):
            linha += instancia[index]
            index += 1

        index += 1
        coluna = ""
        while(instancia[index] != "_"):
            coluna += instancia[index]
            index += 1

        linha = int(linha)
        coluna = int(coluna)
        return linha, coluna

# Modelagem do Mundo de Wumpus para Solver 👾
```
mapeamento = {
  '0_0_wumpus': 0,
  '0_0_poco': 1, 
  '0_0_ouro': 2, 
  '0_0_fedor': 3, 
  '0_0_brisa': 4, 
  '0_0_brilho': 5,
  '0_0_impacto': 6,
  '0_0_grito': 7,
  ...
  'verdade': index
  'falso': -index
}

mapeamento_reverso = {
  0: '0_0_wumpus',
  1: '0_0_poco',
  2: '0_0_ouro',
  3: '0_0_fedor', 
  4: '0_0_brisa',
  5: '0_0_brilho',
  6: '0_0_impacto',
  7: '0_0_grito',
  ...
  index: 'verdade'
  -index: 'falso'
}
```

# Equivalências Lógicas do Mundo de Wumpus Para FNC - Solver 👾
## 0 Wumpus
### 1 regra:
1. pos_wumpus>F => fnc-solve => -pos_wumpus

## 1 Wumpus

## 0 Poço
### 1 regra:
1. pos_poco>F => fnc-solve => -pos_poco

## 1 Poço

## 0 Ouro
### 1 regra:
1. pos_ouro>F => fnc-solve => -pos_ouro

## 1 Ouro
### 1 regra:
1. pos_ouro>V => fnc-solve => pos_ouro
2. pos_ouro > pos_brilho => fnc-solve => -pos_ouro#pos_brilho

## 0 Fedor/Brisa
### 2 regras:
1. pos_>F => fnc-solve => -pos_
2. -pos_>-(c#e#b#d) => fnc-solve => (pos_#-c)&(pos_#-e)&(pos_#-b)&(pos_#-d)

## 1 Fedor/Brisa
### Uma única proposição genérica:
pos_>(c&-e&-b&-d)#(-c&e&-b&-d)#(-c&-e&b&-d)#(-c&-e&-b&d)
### 5 regras de existência:
1. pos_>V => fnc-solve => pos_
2. -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
3. -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
4. -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
5. -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-b#-d)

## 2 Fedor/Brisa
### Uma única proposição genérica:
pos>(c&e)#(c&b)#(c&d)#(e&b)#(e&d)#(b&d)
### 5 regras de existência:
1. pos_>V => fnc-solve => pos_
2. -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
3. -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
4. -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
5. -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)

## 3 Fedor/Brisa
### Uma única proposição genérica:
pos > (-c&e&b&d)#(c&-e&b&d)#(c&e&-b&d)#(c&e&b&-d)
### 5 regras de existência:
1. pos_>V => fnc-solve => pos_
2. -c>(e&b&d) => fnc-solve => (c#e)&(c#b)&(c#d)
3. -e>(c&b&d) => fnc-solve => (e#c)&(e#b)&(e#d)
4. -b>(c&e&d) => fnc-solve => (b#c)&(b#e)&(b#d)
5. -d>(c&e&b) => fnc-solve => (d#c)&(d#e)&(d#b)

## 0 Brilho
### 2 regra:
1. pos_brilho>F => fnc-solve => -pos_brilho
2. -pos_brilho>-pos_ouro => fnc-solve => (pos_brilho#-pos_ouro)

## 1 Brilho
### 2 regra:
1. pos_brilho>V => fnc-solve => pos_brilho
2. pos_brilho>pos_ouro => fnc-solve => (-pos_brilho#pos_ouro)

## 1 OK
### 1 regra:
1. pos_ok>V => fnc-solve => pos_ok

## 0 Impact
### 1 regra:
1. pos_impacto>F => fnc-solver => -pos_impacto

## 1 Impact
1. pos_impacto>V => fnc-solver => pos_impacto

## 0 Grito
## 1 Grito

In [ ]:
class Solver():
  def __init__(self, x, y):
    self.glucose = Glucose3()
    self.mapeamento = {}
    self.mapeamento_reverso = {}
    self.x = x
    self.y = y
    self.mapear(self)
    """
    print(g.solve())
    print(g.get_model())
    """
  
  @staticmethod
  def mapear(self):
    index = 0
    for x in range(0, self.x):
      for y in range(0, self.y):
        self.mapeamento[f"{x}_{y}_wumpus"] = index
        self.mapeamento_reverso[index] = f"{x}_{y}_wumpus"

        self.mapeamento[f"{x}_{y}_poco"] = index + 1
        self.mapeamento_reverso[index + 1] = f"{x}_{y}_poco"

        self.mapeamento[f"{x}_{y}_ouro"] = index + 2
        self.mapeamento_reverso[index + 2] = f"{x}_{y}_ouro"

        self.mapeamento[f"{x}_{y}_fedor"] = index + 3
        self.mapeamento_reverso[index + 3] = f"{x}_{y}_fedor"

        self.mapeamento[f"{x}_{y}_brisa"] = index + 4
        self.mapeamento_reverso[index + 4] = f"{x}_{y}_brisa"

        self.mapeamento[f"{x}_{y}_brilho"] = index + 5
        self.mapeamento_reverso[index + 5] = f"{x}_{y}_brilho"

        index += 6

    self.mapeamento[f"verdade"] = index
    self.mapeamento_reverso[index] = f"verdade"

    self.glucose.add_clause([index])

    self.mapeamento[f"falso"] = -index
    self.mapeamento_reverso[-index] = f"falso"

  def imprimirMapeamento(self):
    print("mapeamento[posição] -> index")
    print(self.mapeamento)
    print("mapeamento_reverso[index] -> posição")
    print(self.mapeamento_reverso)
  
  def perguntarRegra(self, regra):
    regra = self.mapeamento[regra]
    if self.glucose.solve(assumptions=[regra]): return True
    return False
  
  def resolverSolve(self):
    self.glucose.solve()

  def retornarSolve(self):
    self.glucose.get_model()
  
  def imprimirSolve(self):
    print(self.glucose.get_model())

  # Equivalências Lógicas

  def naoPossuiWumpus(self, x, y): 
    # pos_wumpus>F => fnc-solve => -pos_wumpus
    atomo_1 = self.mapeamento[f"{x}_{y}_wumpus"]
    self.glucose.add_clause([-atomo_1])

  def possuiWumpus(self, x, y): pass

  def naoPossuiPoco(self, x, y):
    # pos_poco>F => fnc-solve => -pos_poco
    atomo_1 = self.mapeamento[f"{x}_{y}_poco"]
    self.glucose.add_clause([-atomo_1])
  
  def possuiPoco(self, x, y): pass

  def naoPossuiOuro(self, x, y):
    # pos_ouro>F => fnc-solve => -pos_ouro
    atomo_1 = self.mapeamento[f"{x}_{y}_ouro"]
    self.glucose.add_clause([-atomo_1])

  def possuiOuro(self, x, y):
    # pos_ouro>V => fnc-solve => pos_ouro
    atomo_1 = self.mapeamento[f"{x}_{y}_ouro"]
    self.glucose.add_clause([atomo_1])

    # pos_ouro > pos_brilho => fnc-solve => -pos_ouro#pos_brilho
    atomo_2 = self.mapeamento[f"{x}_{y}_brilho"]
    self.glucose.add_clause([-atomo_1, atomo_2])
  
  def naoPossuiFedor(self, x, y):
    # pos_>F => fnc-solve => -pos_
    atomo_1 = self.mapeamento[f"{x}_{y}_fedor"]
    self.glucose.add_clause([-atomo_1])

    # -pos_>-(c#e#b#d) => fnc-solve => (pos_#-c)&(pos_#-e)&(pos_#-b)&(pos_#-d)
    try:
      atomo_cima = self.mapeamento[f"{x - 1}_{y}_wumpus"]
      self.glucose.add_clause([atomo_1, -atomo_cima])
    except: pass

    try:
      atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_wumpus"]
      self.glucose.add_clause([atomo_1, -atomo_esquerda])
    except: pass

    try:
      atomo_baixo = self.mapeamento[f"{x + 1}_{y}_wumpus"]
      self.glucose.add_clause([atomo_1, -atomo_baixo])
    except: pass

    try:
      atomo_direita = self.mapeamento[f"{x}_{y + 1}_wumpus"]
      self.glucose.add_clause([atomo_1, -atomo_direita])
    except: pass

  def naoPossuiBrisa(self, x, y):
    # pos_>F => fnc-solve => -pos_
    atomo_1 = self.mapeamento[f"{x}_{y}_brisa"]
    self.glucose.add_clause([-atomo_1])

    # -pos_>-(c#e#b#d) => fnc-solve => (pos_#-c)&(pos_#-e)&(pos_#-b)&(pos_#-d)
    try:
      atomo_cima = self.mapeamento[f"{x - 1}_{y}_poco"]
      self.glucose.add_clause([atomo_1, -atomo_cima])
    except: pass

    try:
      atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_poco"]
      self.glucose.add_clause([atomo_1, -atomo_esquerda])
    except: pass

    try:
      atomo_baixo = self.mapeamento[f"{x + 1}_{y}_poco"]
      self.glucose.add_clause([atomo_1, -atomo_baixo])
    except: pass

    try:
      atomo_direita = self.mapeamento[f"{x}_{y + 1}_poco"]
      self.glucose.add_clause([atomo_1, -atomo_direita])
    except: pass

  def possui1Fedor(self, x, y):
    # pos_>V => fnc-solve => pos_
    atomo_1 = self.mapeamento[f"{x}_{y + 1}_fedor"]
    self.glucose.add_clause([atomo_1])

    try: atomo_cima = self.mapeamento[f"{x - 1}_{y}_wumpus"]
    except: atomo_cima = self.mapeamento[f"falso"]
    try: atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_wumpus"]
    except: atomo_esquerda = self.mapeamento[f"falso"]
    try: atomo_baixo = self.mapeamento[f"{x + 1}_{y}_wumpus"]
    except: atomo_baixo = self.mapeamento[f"falso"]
    try: atomo_direita = self.mapeamento[f"{x}_{y + 1}_wumpus"]
    except: atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
    self.glucose.add_clause([atomo_cima, -atomo_esquerda, -atomo_baixo])
    self.glucose.add_clause([atomo_cima, -atomo_esquerda, -atomo_direita])
    self.glucose.add_clause([atomo_cima, -atomo_baixo, -atomo_direita])

    # -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
    self.glucose.add_clause([atomo_esquerda, -atomo_cima, -atomo_baixo])
    self.glucose.add_clause([atomo_esquerda, -atomo_cima, -atomo_direita])
    self.glucose.add_clause([atomo_esquerda, -atomo_baixo, -atomo_direita])

    # -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
    self.glucose.add_clause([atomo_baixo, -atomo_cima, -atomo_esquerda])
    self.glucose.add_clause([atomo_baixo, -atomo_cima, -atomo_direita])
    self.glucose.add_clause([atomo_baixo, -atomo_esquerda, -atomo_direita])

    # -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-b#-d)
    self.glucose.add_clause([atomo_direita, -atomo_cima, -atomo_esquerda])
    self.glucose.add_clause([atomo_direita, -atomo_cima, -atomo_baixo])
    self.glucose.add_clause([atomo_direita, -atomo_baixo, -atomo_direita])

  def possui1Brisa(self, x, y):
    # pos_>V => fnc-solve => pos_
    atomo_1 = self.mapeamento[f"{x}_{y + 1}_brisa"]
    self.glucose.add_clause([atomo_1])

    try: atomo_cima = self.mapeamento[f"{x - 1}_{y}_poco"]
    except: atomo_cima = self.mapeamento[f"falso"]
    try: atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_poco"]
    except: atomo_esquerda = self.mapeamento[f"falso"]
    try: atomo_baixo = self.mapeamento[f"{x + 1}_{y}_poco"]
    except: atomo_baixo = self.mapeamento[f"falso"]
    try: atomo_direita = self.mapeamento[f"{x}_{y + 1}_poco"]
    except: atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&-b&-d)#(-e&b&-d)#(-e&-b&d) => fnc-solve => (c#-e#-b)&(c#-e#-d)&(c#-b#-d)
    self.glucose.add_clause([atomo_cima, -atomo_esquerda, -atomo_baixo])
    self.glucose.add_clause([atomo_cima, -atomo_esquerda, -atomo_direita])
    self.glucose.add_clause([atomo_cima, -atomo_baixo, -atomo_direita])

    # -e>(c&-b&-d)#(-c&b&-d)#(-c&-b&d) => fnc-solve => (e#-c#-b)&(e#-c#-d)&(e#-b#-d)
    self.glucose.add_clause([atomo_esquerda, -atomo_cima, -atomo_baixo])
    self.glucose.add_clause([atomo_esquerda, -atomo_cima, -atomo_direita])
    self.glucose.add_clause([atomo_esquerda, -atomo_baixo, -atomo_direita])

    # -b>(c&-e&-d)#(-c&e&-d)#(-c&-e&d) => fnc-solve => (b#-c#-e)&(b#-c#-d)&(b#-e#-d)
    self.glucose.add_clause([atomo_baixo, -atomo_cima, -atomo_esquerda])
    self.glucose.add_clause([atomo_baixo, -atomo_cima, -atomo_direita])
    self.glucose.add_clause([atomo_baixo, -atomo_esquerda, -atomo_direita])

    # -d>(c&-e&-b)#(-c&e&-b)#(-c&-e&b) => fnc-solve => (d#-c#-e)&(d#-c#-b)&(d#-b#-d)
    self.glucose.add_clause([atomo_direita, -atomo_cima, -atomo_esquerda])
    self.glucose.add_clause([atomo_direita, -atomo_cima, -atomo_baixo])
    self.glucose.add_clause([atomo_direita, -atomo_baixo, -atomo_direita])
     
  def possui2Fedor(self, x, y):
    # pos_>V => fnc-solve => pos_
    atomo_1 = self.mapeamento[f"{x}_{y + 1}_fedor"]
    self.glucose.add_clause([atomo_1])

    try: atomo_cima = self.mapeamento[f"{x - 1}_{y}_wumpus"]
    except: atomo_cima = self.mapeamento[f"falso"]
    try: atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_wumpus"]
    except: atomo_esquerda = self.mapeamento[f"falso"]
    try: atomo_baixo = self.mapeamento[f"{x + 1}_{y}_wumpus"]
    except: atomo_baixo = self.mapeamento[f"falso"]
    try: atomo_direita = self.mapeamento[f"{x}_{y + 1}_wumpus"]
    except: atomo_direita = self.mapeamento[f"falso"]

    # -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
    self.glucose.add_clause([atomo_cima, atomo_esquerda, atomo_baixo])
    self.glucose.add_clause([atomo_cima, atomo_esquerda, atomo_direita])
    self.glucose.add_clause([atomo_cima, atomo_baixo, atomo_direita])

    # -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
    self.glucose.add_clause([atomo_esquerda, atomo_cima, atomo_baixo])
    self.glucose.add_clause([atomo_esquerda, atomo_cima, atomo_direita])
    self.glucose.add_clause([atomo_esquerda, atomo_baixo, atomo_direita])

    # -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
    self.glucose.add_clause([atomo_baixo, atomo_cima, atomo_esquerda])
    self.glucose.add_clause([atomo_baixo, atomo_cima, atomo_direita])
    self.glucose.add_clause([atomo_baixo, atomo_esquerda, atomo_direita])

    # -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)
    self.glucose.add_clause([atomo_direita, atomo_cima, atomo_esquerda])
    self.glucose.add_clause([atomo_direita, atomo_cima, atomo_baixo])
    self.glucose.add_clause([atomo_direita, atomo_esquerda, atomo_baixo])

  def possui2Brisa(self, x, y):
    # pos_>V => fnc-solve => pos_
    atomo_1 = self.mapeamento[f"{x}_{y + 1}_brisa"]
    self.glucose.add_clause([atomo_1])

    try: atomo_cima = self.mapeamento[f"{x - 1}_{y}_poco"]
    except: atomo_cima = self.mapeamento[f"falso"]
    try: atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_poco"]
    except: atomo_esquerda = self.mapeamento[f"falso"]
    try: atomo_baixo = self.mapeamento[f"{x + 1}_{y}_poco"]
    except: atomo_baixo = self.mapeamento[f"falso"]
    try: atomo_direita = self.mapeamento[f"{x}_{y + 1}_poco"]
    except: atomo_direita = self.mapeamento[f"falso"]

    # -c>((e&b)#(e&d)#(b&d)) => fnc-solve => (c#e#b)&(c#e#d)&(c#b#d)
    self.glucose.add_clause([atomo_cima, atomo_esquerda, atomo_baixo])
    self.glucose.add_clause([atomo_cima, atomo_esquerda, atomo_direita])
    self.glucose.add_clause([atomo_cima, atomo_baixo, atomo_direita])

    # -e>((c&b)&(c&d)&(b&d)) => fnc-solve => (e#c#b)&(e#c#d)&(e#b#d)
    self.glucose.add_clause([atomo_esquerda, atomo_cima, atomo_baixo])
    self.glucose.add_clause([atomo_esquerda, atomo_cima, atomo_direita])
    self.glucose.add_clause([atomo_esquerda, atomo_baixo, atomo_direita])

    # -b>((c&e)#(c&d)#(e&d)) => fnc-solve => (b#c#e)&(b#c#d)&(b#e#d)
    self.glucose.add_clause([atomo_baixo, atomo_cima, atomo_esquerda])
    self.glucose.add_clause([atomo_baixo, atomo_cima, atomo_direita])
    self.glucose.add_clause([atomo_baixo, atomo_esquerda, atomo_direita])

    # -d>((c&e)#(c&b)#(e&b)) => fnc-solve => (d#c#e)&(d#c#b)&(d#e#b)
    self.glucose.add_clause([atomo_direita, atomo_cima, atomo_esquerda])
    self.glucose.add_clause([atomo_direita, atomo_cima, atomo_baixo])
    self.glucose.add_clause([atomo_direita, atomo_esquerda, atomo_baixo])

  def possui3Fedor(self, x, y):
    # pos_>V => fnc-solve => pos_
    atomo_1 = self.mapeamento[f"{x}_{y + 1}_fedor"]
    self.glucose.add_clause([atomo_1])

    try: atomo_cima = self.mapeamento[f"{x - 1}_{y}_wumpus"]
    except: atomo_cima = self.mapeamento[f"falso"]
    try: atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_wumpus"]
    except: atomo_esquerda = self.mapeamento[f"falso"]
    try: atomo_baixo = self.mapeamento[f"{x + 1}_{y}_wumpus"]
    except: atomo_baixo = self.mapeamento[f"falso"]
    try: atomo_direita = self.mapeamento[f"{x}_{y + 1}_wumpus"]
    except: atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&b&d) => fnc-solve => (c#e)&(c#b)&(c#d)
    self.glucose.add_clause([atomo_cima, atomo_esquerda])
    self.glucose.add_clause([atomo_cima, atomo_baixo])
    self.glucose.add_clause([atomo_cima, atomo_direita])

    # -e>(c&b&d) => fnc-solve => (e#c)&(e#b)&(e#d)
    self.glucose.add_clause([atomo_esquerda, atomo_cima])
    self.glucose.add_clause([atomo_esquerda, atomo_baixo])
    self.glucose.add_clause([atomo_esquerda, atomo_direita])

    # -b>(c&e&d) => fnc-solve => (b#c)&(b#e)&(b#d)
    self.glucose.add_clause([atomo_baixo, atomo_cima])
    self.glucose.add_clause([atomo_baixo, atomo_esquerda])
    self.glucose.add_clause([atomo_baixo, atomo_direita])

    # -d>(c&e&b) => fnc-solve => (d#c)&(d#e)&(d#b)
    self.glucose.add_clause([atomo_direita, atomo_cima])
    self.glucose.add_clause([atomo_direita, atomo_esquerda])
    self.glucose.add_clause([atomo_direita, atomo_baixo])
  
  def possui3Brisa(self, x, y):
    # pos_>V => fnc-solve => pos_
    atomo_1 = self.mapeamento[f"{x}_{y + 1}_brisa"]
    self.glucose.add_clause([atomo_1])

    try: atomo_cima = self.mapeamento[f"{x - 1}_{y}_poco"]
    except: atomo_cima = self.mapeamento[f"falso"]
    try: atomo_esquerda = self.mapeamento[f"{x}_{y - 1}_poco"]
    except: atomo_esquerda = self.mapeamento[f"falso"]
    try: atomo_baixo = self.mapeamento[f"{x + 1}_{y}_poco"]
    except: atomo_baixo = self.mapeamento[f"falso"]
    try: atomo_direita = self.mapeamento[f"{x}_{y + 1}_poco"]
    except: atomo_direita = self.mapeamento[f"falso"]

    # -c>(e&b&d) => fnc-solve => (c#e)&(c#b)&(c#d)
    self.glucose.add_clause([atomo_cima, atomo_esquerda])
    self.glucose.add_clause([atomo_cima, atomo_baixo])
    self.glucose.add_clause([atomo_cima, atomo_direita])

    # -e>(c&b&d) => fnc-solve => (e#c)&(e#b)&(e#d)
    self.glucose.add_clause([atomo_esquerda, atomo_cima])
    self.glucose.add_clause([atomo_esquerda, atomo_baixo])
    self.glucose.add_clause([atomo_esquerda, atomo_direita])

    # -b>(c&e&d) => fnc-solve => (b#c)&(b#e)&(b#d)
    self.glucose.add_clause([atomo_baixo, atomo_cima])
    self.glucose.add_clause([atomo_baixo, atomo_esquerda])
    self.glucose.add_clause([atomo_baixo, atomo_direita])

    # -d>(c&e&b) => fnc-solve => (d#c)&(d#e)&(d#b)
    self.glucose.add_clause([atomo_direita, atomo_cima])
    self.glucose.add_clause([atomo_direita, atomo_esquerda])
    self.glucose.add_clause([atomo_direita, atomo_baixo])
    
  def naoPossuiBrilho(self, x, y):
    # pos_brilho>F => fnc-solve => -pos_brilho
    atomo_1 = self.mapeamento[f"{x}_{y}_brilho"]
    self.glucose.add_clause([-atomo_1])

    # -pos_brilho>-pos_ouro => fnc-solve => (pos_brilho#-pos_ouro)
    atomo_2 = self.mapeamento[f"{x}_{y}_ouro"]
    self.glucose.add_clause([atomo_1, -atomo_2])
  
  def possuiBrilho(self, x, y):
    # pos_brilho>V => fnc-solve => pos_brilho
    atomo_1 = self.mapeamento[f"{x}_{y}_brilho"]
    self.glucose.add_clause([atomo_1])

    # pos_brilho>pos_ouro => fnc-solve => (-pos_brilho#pos_ouro)
    atomo_2 = self.mapeamento[f"-{x}_{y}_ouro"]
    self.glucose.add_clause([-atomo_1, atomo_2])

  def possuioOK(self, x, y):    
    # pos_ok>V => fnc-solve => pos_ok
    atomo_1 = self.mapeamento[f"{x}_{y}_ok"]
    self.glucose.add_clause([atomo_1])

In [ ]:
solver = Solver(6, 6)
solver.imprimirMapeamento()

mapeamento[posição] -> index
{'0_0_wumpus': 0, '0_0_poco': 1, '0_0_ouro': 2, '0_0_fedor': 3, '0_0_brisa': 4, '0_0_brilho': 5, '0_1_wumpus': 6, '0_1_poco': 7, '0_1_ouro': 8, '0_1_fedor': 9, '0_1_brisa': 10, '0_1_brilho': 11, '0_2_wumpus': 12, '0_2_poco': 13, '0_2_ouro': 14, '0_2_fedor': 15, '0_2_brisa': 16, '0_2_brilho': 17, '0_3_wumpus': 18, '0_3_poco': 19, '0_3_ouro': 20, '0_3_fedor': 21, '0_3_brisa': 22, '0_3_brilho': 23, '0_4_wumpus': 24, '0_4_poco': 25, '0_4_ouro': 26, '0_4_fedor': 27, '0_4_brisa': 28, '0_4_brilho': 29, '0_5_wumpus': 30, '0_5_poco': 31, '0_5_ouro': 32, '0_5_fedor': 33, '0_5_brisa': 34, '0_5_brilho': 35, '1_0_wumpus': 36, '1_0_poco': 37, '1_0_ouro': 38, '1_0_fedor': 39, '1_0_brisa': 40, '1_0_brilho': 41, '1_1_wumpus': 42, '1_1_poco': 43, '1_1_ouro': 44, '1_1_fedor': 45, '1_1_brisa': 46, '1_1_brilho': 47, '1_2_wumpus': 48, '1_2_poco': 49, '1_2_ouro': 50, '1_2_fedor': 51, '1_2_brisa': 52, '1_2_brilho': 53, '1_3_wumpus': 54, '1_3_poco': 55, '1_3_ouro': 56, '1_3_fedor':

In [ ]:
class BD:
  def __init__(self):
    self.conn = sqlite3.connect("base_de_conhecimento")
    self.cursor = self.conn.cursor()
    self.executarComando(self, """
      create table if not exists conhecimento (
        tempo integer primary key autoincrement,
        sentenca varchar not null
      )
    """)
    self.executarComando(self, """
      create table if not exists raciocinio (
        tempo integer primary key autoincrement,
        sentenca varchar not null
      )
    """)
  
  @staticmethod
  def executarComando(self, comando):
    try:
      self.cursor.execute(comando)
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
  
  def imprimirConhecimento(self):
    try:
      self.cursor.execute(f"select * from conhecimento")
      sentencas = self.cursor.fetchall()
      for sentenca in sentencas:
        print(sentenca[0], sentenca[1])
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def imprimirRaciocinio(self):
    try:
      self.cursor.execute(f"select * from raciocinio")
      sentencas = self.cursor.fetchall()
      for sentenca in sentencas:
        print(sentenca[0], sentenca[1])
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def retornarConhecimentoCompleto(self):
    try:
      self.cursor.execute(f"select * from conhecimento")
      sentencas = self.cursor.fetchall()
      return sentencas
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False

  def retornarConhecimento(self, tempo):
    try:
      self.cursor.execute(f"select sentenca from conhecimento where tempo = {tempo}")
      sentenca = self.cursor.fetchall()[0][1]
      return sentenca
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False

  def retornarRaciocinio(self):
    try:
      self.cursor.execute(f"select * from raciocinio")
      sentencas = self.cursor.fetchall()
      return sentencas
    except sqlite3.Error as erro:
      print(f"Error: {erro}")
      return False

  def retornarRaciocinio(self, tempo): pass

  def inserirConhecimento(self, tempo, sentenca):
    try:
      self.cursor.execute(f"insert into conhecimento (tempo, sentenca) values ('{tempo}', '{sentenca}')")
      self.conn.commit()
    except sqlite3.Error as erro:
      print(f"Error: {erro}")

  def inserirRaciocinio(self, tempo, sentenca): pass

In [ ]:
bd = BD()
bd.imprimirConhecimento()
conhecimento = bd.retornarTConhecimento()
print(conhecimento[0][1])

AttributeError: ignored

# Agente

In [ ]:
class Agente():
  def __init__(self, mapa, solver):
    self.mapa = mapa
    self.solver = solver
    self.percepcoes = ()
    self.x = 0
    self.y = 0
    self.orientacao = 0

  @staticmethod
  def receberSensor(self, x, y):
    return self.mapa.retornarStatusDaPosica(x, y)

  @staticmethod
  def inferirPercepcoes(self):
    wumpus = self.percepcoes[0][0]
    poco = self.percepcoes[0][1]
    ouro = self.percepcoes[0][2]
    fedor = self.percepcoes[1][0]
    brisa = self.percepcoes[1][1]
    brilho = self.percepcoes[1][2]
    grito = self.percepcoes[1][3]
    # imapacto = self.percepcoes[1][4]

    if wumpus: self.solver.possuiWumpus(x, y)
    else: self.solver.naoPossuiWumpus(x, y)

    if poco: self.solver.possuiPoco(x, y)
    else: self.solver.naoPossuiPoco(x, y)

    if ouro: self.solver.possuiOuro(x, y)
    else: self.solver.naoPossuiOuro(x, y)

    if fedor == 1: self.solver.possui1Fedor(x, y)
    elif fedor == 2: self.solver.possui2Fedor(x, y)
    elif fedor == 3: self.solver.possui3Fedor(x, y)
    else: self.solver.naoPossuiFedor(x, y)

    if brisa == 1: self.solver.possui1Brisa(x, y)
    elif brisa == 2: self.solver.possui2Brisa(x, y)
    elif brisa == 3: self.solver.possui3Brisa(x, y)
    else: self.solver.naoPossuiBrisa(x, y)

    if brilho: self.solver.possuiBrilho(x, y)
    else: self.solver.naoPossuiBrilho(x, y)

    # Impacto

  @staticmethod
  def avancar(self):
    if self.orientacao == 0: self.y += 1
    elif self.orientacao == 90: self.x -= 1
    elif self.orientacao == 180: self.y -= 1
    elif self.orientacao == 270: self.x += 1
  
  @staticmethod
  def girarParaEsquerda(self):
    self.orientacao -= 90
    if self.orientacao == -90: self.orientacao = 90
  
  @staticmethod
  def girarParaDireita(self):
    self.orientacao += 90
    if self.orientacao == 360: self.orientacao = 0

# Main

In [ ]:
mapa = Mapa(linhas = 4, colunas = 4, quantidade_wumpus = 1, quantidade_pocos = 2, quantidade_ouro = 2)
mapa.imprimirMapa()
solver = Solver(x = 4, y = 4)

  ok      ok      ok    ok
 ouro   1fedor   ouro   ok
1fedor  wumpus  1fedor  ok
  ok    1fedor    ok    ok

